In [53]:
import pandas as pd
import numpy as np
import geonamescache
from unidecode import unidecode
import re

In [54]:
df_headlines = pd.read_csv(r"C:\Users\SL\Desktop\discovering-disease-outbreaks-base-master\data\headlines.txt", names =['headline'], sep="\n")

In [55]:
df_headlines.sample(20)

,headline
334,Chickenpox Hits Simpsonville
591,Manchester Residents Recieve Influenza vaccine
52,Zika symtomps spotted in Key West
609,Zika Reported in Ilopango
218,Zika spreads to Daytona Beach
4,Dallas man comes down with case of Zika
232,How to Avoid Hepatitis B in Waldorf
63,Carnival under threat in Rio De Janeiro due to...
264,Tuberculosis Outbreak in Hartford
194,Pittsburgh Residents Recieve Norovirus vaccine


In [56]:
df_headlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 1 columns):
headline    650 non-null object
dtypes: object(1)
memory usage: 5.2+ KB


In [57]:
df_headlines.headline = df_headlines.headline.astype(str)
df_headlines.shape

(650, 1)

In [58]:
gc = geonamescache.GeonamesCache()

countries = gc.get_countries()
df_countries = pd.DataFrame(countries).T.reset_index(drop=True)

cities = gc.get_cities()
df_cities = pd.DataFrame(cities).T.reset_index(drop=True)

In [59]:
df_cities.head()

,geonameid,name,latitude,longitude,countrycode,population,timezone,admin1code
0,3041563,Andorra la Vella,42.5078,1.52109,AD,20430,Europe/Andorra,07
1,290594,Umm Al Quwain City,25.5647,55.5552,AE,62747,Asia/Dubai,07
2,291074,Ras Al Khaimah City,25.7895,55.9432,AE,351943,Asia/Dubai,05
3,291580,Zayed City,23.6542,53.7052,AE,63482,Asia/Dubai,01
4,291696,Khawr Fakkān,25.3313,56.342,AE,40677,Asia/Dubai,06


In [60]:
df_cities[df_cities.name =='Beli']

,geonameid,name,latitude,longitude,countrycode,population,timezone,admin1code
15482,2347330,Beli,7.85868,10.9719,NG,17467,Africa/Lagos,43


In [61]:
 df_countries.head()

,geonameid,name,iso,iso3,isonumeric,fips,continentcode,capital,areakm2,population,tld,currencycode,currencyname,phone,postalcoderegex,languages,neighbours
0,3041565,Andorra,AD,AND,20,AN,EU,Andorra la Vella,468,84000,.ad,EUR,Euro,376,^(?:AD)*(\d{3})$,ca,"ES,FR"
1,290557,United Arab Emirates,AE,ARE,784,AE,AS,Abu Dhabi,82880,4975593,.ae,AED,Dirham,971,,"ar-AE,fa,en,hi,ur","SA,OM"
2,1149361,Afghanistan,AF,AFG,4,AF,AS,Kabul,647500,29121286,.af,AFN,Afghani,93,,"fa-AF,ps,uz-AF,tk","TM,CN,IR,TJ,PK,UZ"
3,3576396,Antigua and Barbuda,AG,ATG,28,AC,NA,St. John's,443,86754,.ag,XCD,Dollar,+1-268,,en-AG,
4,3573511,Anguilla,AI,AIA,660,AV,NA,The Valley,102,13254,.ai,XCD,Dollar,+1-264,,en-AI,


In [62]:
df_cities.shape

(24336, 8)

In [63]:
df_countries.shape

(252, 17)

In [64]:
df_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24336 entries, 0 to 24335
Data columns (total 8 columns):
geonameid      24336 non-null object
name           24336 non-null object
latitude       24336 non-null object
longitude      24336 non-null object
countrycode    24336 non-null object
population     24336 non-null object
timezone       24336 non-null object
admin1code     24336 non-null object
dtypes: object(8)
memory usage: 1.5+ MB


In [65]:
cities = df_cities.name = df_cities.name.astype(str)
countries = df_countries.name = df_countries.name.astype(str)

Normalizing headlines and city/country names by removing the accent marks:

In [66]:
for i in range(df_headlines.shape[0]):
    df_headlines.headline.iloc[i] = unidecode(df_headlines.headline.iloc[i])
    
for i in range(len(cities)):
    cities[i] = unidecode(cities[i])
    
for i in range(len(countries)):
    countries[i] = unidecode(countries[i])

In [67]:
'Khawr Fakkan' in cities.values

True

In [68]:
'Khawr Fakkān' in cities.values

False

REGEX

In [69]:
df_headlines.shape[0]

650

In [70]:
cities_found = []
countries_found = []
for i in np.arange(df_headlines.shape[0]):
    cities_found.append(None)
    countries_found.append(None)

for name in cities:
    regexp = re.compile(name)
    for i, headline in enumerate(df_headlines.headline):
           if regexp.search(headline):
                cities_found[i] = name

In [71]:
how_many_words = [len(city.split(' ')) for city in cities]
how_long_name = [len(city) for city in cities]

In [72]:
modified_cities = pd.DataFrame(list(cities.values), columns=['city'])
modified_cities['how_many_words'] = how_many_words
modified_cities['how_long_name'] = how_long_name
modified_cities = modified_cities.sort_values(by=['how_many_words', 'how_long_name'], ascending=True)

In [73]:
modified_cities.head(3)

,city,how_many_words,how_long_name
1063,Po,1,2
8325,Wa,1,2
8358,Ho,1,2


In [74]:
for name in modified_cities.city:
    regexp = re.compile(name)
    for i, headline in enumerate(df_headlines.headline):
           if regexp.search(headline):
                cities_found[i] = name
cities_found

['Miami',
 'New York City',
 'Miami Beach',
 'Recife',
 'Dallas',
 'Trinidad',
 'Houston',
 'Geneve',
 'Atlanta',
 'Sao Paulo',
 'Brownsville',
 'St. Louis',
 'San Juan',
 'Galveston',
 'Manila',
 'Iloilo',
 'Los Angeles',
 None,
 'Orlando',
 None,
 'Chicago',
 'Tampa',
 'Flint',
 'Baltimore',
 'London',
 'Ho Chi Minh City',
 'Philadelphia',
 'Boston',
 'Paris',
 'San Diego',
 'Bangkok',
 'Beijing',
 'Salvador',
 'Kuala Lumpur',
 'Yangon',
 'Tallahassee',
 'San Francisco',
 'Bethesda',
 'Townsville',
 'Mandaluyong City',
 'Santa Rosa',
 'San Salvador',
 'Cleveland',
 'Austin',
 'Piracicaba',
 'Lima',
 'Toronto',
 'Bogota',
 'Sig',
 'Brisbane',
 'Dakar',
 'Havana',
 'Key West',
 'Vancouver',
 'Seattle',
 'Nashville',
 'Saint Croix',
 'Fort Collins',
 'Klang',
 'Guatemala City',
 'Madison',
 'Sarasota',
 'Entebbe',
 None,
 'Brasilia',
 'Jacksonville',
 'Shenzhen',
 'Caracas',
 'Quezon City',
 'Sydney',
 'Mumbai',
 'Pune',
 'Hanoi',
 None,
 'Silver Spring',
 'Delhi',
 'Hillsboro',
 'Beliz

In [75]:
how_many_words = [len(country.split(' ')) for country in countries]
how_long_name = [len(country) for country in countries]

In [76]:
modified_countries = pd.DataFrame(list(countries.values), columns=['country'])
modified_countries['how_many_words'] = how_many_words
modified_countries['how_long_name'] = how_long_name
modified_countries = modified_countries.sort_values(by=['how_many_words', 'how_long_name'], ascending=True)

In [77]:
for name in modified_countries.country:
    regexp = re.compile(name)
    for i, headline in enumerate(df_headlines.headline):
           if regexp.search(headline):
                countries_found[i] = regexp.pattern
countries_found

[None,
 None,
 None,
 'Brazil',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'Vietnam',
 None,
 None,
 None,
 None,
 'Thailand',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'Brazil',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'Malaysia',
 'Guatemala',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'Belize',
 'Brazil',
 None,
 None,
 None,
 None,
 'Mexico',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'Malaysia',
 None,
 None,
 'Malaysia',
 None,
 None,
 None,
 'Hong Kong',
 None,
 None,

In [78]:
df_headlines['countries'] = countries_found
df_headlines['cities'] = cities_found
df = df_headlines

In [81]:
df.sample(50)

,headline,countries,cities
403,Huzhou Residents Receive Hepatitis B vaccine,None,Huzhou
94,New Zika Case Confirmed in Belo Horizonte,None,Belo Horizonte
501,More people in Yokohama are infected with Noro...,None,Yokohama
186,More Patients in Cambridge are Getting Diagnos...,None,Cambridge
223,Rumors about Hepatitis D spreading in Albany h...,None,Albany
392,Pedro Juan Caballero Encounters Severe Symptom...,None,Pedro Juan Caballero
571,Fayetteville authorities confirmed the spread ...,None,Fayetteville
386,More Zika patients reported in Carrollton,None,Carrollton
643,Outbreak of Zika in Portoviejo,None,Portoviejo
598,Schools in Coamo Closed Due to Rhinovirus Outb...,None,None
